In [ ]:
from autovc.utils.dataloader import TrainDataLoader
from autovc.utils.model_loader import load_models
# from autovc.auto_encoder.model_vc import Generator
# from autovc.speaker_encoder.model import SpeakerEncoder

# 


# model = Generator()
# model.load_model('models/AutoVC/AutoVC_SMK.pt', device = 'cpu')

# speaker_encoder = SpeakerEncoder()
# speaker_encoder.load_model('models/SpeakerEncoder/SpeakerEncoder.pt')

model, speaker_encoder = load_models(
    model_types= ["auto_encoder", "speaker_encoder"],
    model_paths= ['models/AutoVC/AutoVC_SMK.pt', 'models/SpeakerEncoder/SpeakerEncoder.pt']
)

dataset = TrainDataLoader(data_dir_path = 'data/samples', speaker_encoder = speaker_encoder)
dataloader = dataset.get_dataloader(batch_size = 2, shuffle = True)

model.learn(dataloader, n_epochs = 2)


: 

In [ ]:
from autovc.speaker_encoder.model import SpeakerEncoder
from autovc.speaker_encoder.utils import wav_to_mel_spectrogram, preprocess_wav
import torch
speaker_encoder = SpeakerEncoder()
speaker_encoder.load_model('Models/SpeakerEncoder/SpeakerEncoder.pt')
mels = [speaker_encoder(torch.from_numpy(wav_to_mel_spectrogram(preprocess_wav(wav))).unsqueeze(0)) for wav in ["data/samples/chooped7.wav", "data/samples/mette_183.wav"]]


In [ ]:
speaker_encoder.loss(torch.stack(mels))


In [ ]:
from autovc.utils.audio import get_mel_frames, audio_to_melspectrogram
import torch
import numpy as np
import matplotlib.pyplot as plt
from autovc.utils.model_loader import load_model
from torch.nn.functional import pad
import soundfile as sf
from autovc.utils.audio import remove_noise
AE = load_model('auto_encoder', 'models/AutoVC/AutoVC_SMK.pt')
SE = load_model('speaker_encoder', 'models/SpeakerEncoder/SpeakerEncoder.pt')
vocoder = load_model('vocoder', 'models/WaveRNN/WaveRNN_Pretrained.pyt')
N = 1000
source = 'data/samples/hilde_301.wav'
target = 'data/samples/HaegueYang_5.wav'
min_pad_coverage = 0.1
overlap = 0.5
frames = get_mel_frames(source,
                        audio_to_melspectrogram, 
                        sr = 22050, 
                        mel_window_step             = 12.5, 
                        order                       = 'MF', 
                        partial_utterance_n_frames  = N, 
                        min_pad_coverage            = min_pad_coverage, 
                        overlap                     = overlap,
                        )

X = torch.stack(frames)
c_source = SE.embed_utterance(source).unsqueeze(0).expand(X.size(0),-1)
c_target = SE.embed_utterance(target).unsqueeze(0).expand(X.size(0),-1)


out, post_out, content_codes = AE(X, c_source, c_target)


In [ ]:

frames = list(post_out)
M = len(frames)
T = int(N * (1-overlap))
for i in range(M):
    frames[i] = pad(frames[i], (i * T, (M-i-1) * T) , mode = 'constant', value = torch.nan)

X = torch.stack(frames)
X_paste = X.nanmean(axis = 0)
X_paste.shape

In [ ]:
waveform = vocoder.generate(X_paste.unsqueeze(0))

In [ ]:

wave = np.asarray(waveform)
wave = remove_noise(wave, 22050)
sf.write('chop_and_past_00p_overlap.wav', wave, samplerate =22050)

In [ ]:
SE.state_dict()['test'] = 2

torch.save( {'step':0, 'model_state': SE.state_dict(), 'speakers':{'hilde' : torch.randn((1,256)), 'yang':torch.randn((1,256))}}, 'models/SpeakerEncoder/SpeakerEncoder2.pt')

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('deep_voice_inc/SpeakerEncoder2/model_20220117.pt:v47', type='SpeakerEncoder')
artifact_dir = artifact.download()


In [ ]:

from autovc.utils.model_loader import load_model

SE = load_model('speaker_encoder', 'artifacts/speaker1/model_20220117.pt')

In [ ]:
SE.learn_speaker('hilde')